# Импорты

In [2]:
import pandas as pd 
import numpy as np 
from catboost import CatBoostClassifier # Катбустер
from sklearn.model_selection import train_test_split # Для разделения выборок
from sklearn.metrics import log_loss, accuracy_score # Метрики
import ast # Abstract Syntax Trees
from sklearn.ensemble import RandomForestClassifier # Рандомный лес
from sklearn.linear_model import LogisticRegression # Логистическая регрессия
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score # Метрики

# Подгрузка

In [3]:
geo_info = pd.read_csv('./VK_data/geo_info.csv', sep=';')
referer_vectors = pd.read_csv('./VK_data/referer_vectors.csv', sep=';')
test_users = pd.read_csv('./VK_data/test_users.csv', sep=';')
test = pd.read_csv('./VK_data/test.csv', sep=';')
train_labels = pd.read_csv('./VK_data/train_labels.csv', sep=';')
train = pd.read_csv('./VK_data/train.csv', sep=';')

# Работа с датафреймами

In [4]:
# Объединяем датафреймы
train = train.merge(geo_info, on='geo_id', how='left')
train = train.merge(train_labels, on='user_id', how='left')
train = train.merge(referer_vectors, on='referer', how='left')

In [5]:
# Извлечение информации о браузере, ОС и их версиях
def safe_literal_eval(x):
    if pd.isna(x):
        return {}
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return {}

train['parsed_user_agent'] = train['user_agent'].apply(safe_literal_eval)
train['browser'] = train['parsed_user_agent'].apply(lambda x: x.get('browser', None))
train['browser_version'] = train['parsed_user_agent'].apply(lambda x: x.get('browser_version', None))
train['os'] = train['parsed_user_agent'].apply(lambda x: x.get('os', None))
train['os_version'] = train['parsed_user_agent'].apply(lambda x: x.get('os_version', None))

train.drop(columns=['parsed_user_agent'], inplace=True)

In [6]:
# Извлечем сам домен
def extract_domain(url):
    # Разделяем URL на две части: до 'https://' и после '/'
    parts = url.split('://')
    if len(parts) < 2:
        return None
    
    domain_part = parts[1].split('/')[0]
    return domain_part

# Применение к колонке
train['domain'] = train['referer'].apply(extract_domain)

In [7]:
referer_vectors.dtypes # Посмотрим на тип данных

component0     int64
component1     int64
component2     int64
component3     int64
component4     int64
component5     int64
component6     int64
component7     int64
component8     int64
component9     int64
referer       object
dtype: object

In [8]:
# Список колонок векторов
vector_columns = ['component0', 'component1', 'component2', 'component3', 'component4',
       'component5', 'component6', 'component7', 'component8', 'component9']

In [9]:
# Смена типа данных
for col in vector_columns:
    train[col] = train[col].astype(str)

In [10]:
train.dtypes # Проверим что смена типа данных прошла успешно

request_ts           int64
user_id             object
referer             object
geo_id               int64
user_agent          object
country_id          object
region_id           object
timezone_id         object
target             float64
component0          object
component1          object
component2          object
component3          object
component4          object
component5          object
component6          object
component7          object
component8          object
component9          object
browser             object
browser_version     object
os                  object
os_version          object
domain              object
dtype: object

In [11]:
train = train.dropna()

In [12]:
# Сменим тип данных с float
train['target'] = train['target'].astype(int)

In [13]:
# Избавимся от ненужных теперь столбцов
train = train.drop(['geo_id', 'user_agent', 'referer'], axis=1)

In [14]:
# train = train.dropna() 

In [15]:
train.head()

,request_ts,user_id,country_id,region_id,timezone_id,target,component0,component1,component2,component3,...,component5,component6,component7,component8,component9,browser,browser_version,os,os_version,domain
0,1701011363,fb858e8e0a2bec074450eaf94b627fd3,c31b4e,470e75,f6155e,0,11731,4045,22213,-1184,...,9381,-3496,-3120,-899,16817,Chrome Mobile,119.0.0,Android,10,9b48ee5
1,1700986581,46a5f128fd569c764a92c2eaa788095e,c31b4e,44520b,e56e80,0,11731,4045,22213,-1184,...,9381,-3496,-3120,-899,16817,Chrome Mobile,111.0.0,Android,10,9b48ee5
2,1701011071,5a74e9ac53ffb21a20cce117c0ad77ba,c31b4e,616bb9,af47f1,0,12498,2451,10304,-6380,...,3106,-2188,10573,3347,21870,Yandex Browser,20.12.5,Android,11,9634fd0
3,1700992803,af735816ca19115431ae3d89518c8c91,c31b4e,3c9dca,e56e80,0,11731,4045,22213,-1184,...,9381,-3496,-3120,-899,16817,Chrome Mobile,119.0.0,Android,10,9b48ee5
4,1701021666,364f0ae0a3f29a685c4fb5bae6033b9a,c31b4e,776e76,10b7947,0,11731,4045,22213,-1184,...,9381,-3496,-3120,-899,16817,Yandex Browser,18.11.1,Android,4.4.4,9b48ee5


# Предобработка

In [16]:
object_columns = train.select_dtypes(include=['object']).columns
object_columns

Index(['user_id', 'country_id', 'region_id', 'timezone_id', 'component0',
       'component1', 'component2', 'component3', 'component4', 'component5',
       'component6', 'component7', 'component8', 'component9', 'browser',
       'browser_version', 'os', 'os_version', 'domain'],
      dtype='object')

In [17]:
train[object_columns] = train[object_columns].apply(pd.Categorical)

In [18]:
train[object_columns] = train[object_columns].apply(lambda x: x.cat.codes)

In [19]:
print(train.dtypes)

request_ts         int64
user_id            int32
country_id          int8
region_id          int16
timezone_id         int8
target             int32
component0         int16
component1         int16
component2         int16
component3         int16
component4         int16
component5         int16
component6         int16
component7         int16
component8         int16
component9         int32
browser             int8
browser_version    int16
os                  int8
os_version         int16
domain             int16
dtype: object


In [20]:
train.head()

,request_ts,user_id,country_id,region_id,timezone_id,target,component0,component1,component2,component3,...,component5,component6,component7,component8,component9,browser,browser_version,os,os_version,domain
0,1701011363,449437,8,116,55,0,5230,24421,17110,1615,...,25684,3806,5318,9947,18956,7,180,0,1,2999
1,1700986581,126428,8,111,50,0,5230,24421,17110,1615,...,25684,3806,5318,9947,18956,7,117,0,1,2999
2,1701011071,161801,8,160,23,0,6048,22542,8914,10952,...,18744,2351,12292,20284,24417,52,528,0,47,2744
3,1700992803,313347,8,99,50,0,5230,24421,17110,1615,...,25684,3806,5318,9947,18956,7,180,0,1,2999
4,1701021666,97217,8,200,5,0,5230,24421,17110,1615,...,25684,3806,5318,9947,18956,52,437,0,191,2999


# Разделение выборок

In [21]:
X = train.drop(columns=['target'])
y = train['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
print(X_train.dtypes)

request_ts         int64
user_id            int32
country_id          int8
region_id          int16
timezone_id         int8
component0         int16
component1         int16
component2         int16
component3         int16
component4         int16
component5         int16
component6         int16
component7         int16
component8         int16
component9         int32
browser             int8
browser_version    int16
os                  int8
os_version         int16
domain             int16
dtype: object


In [23]:
X_train.dtypes

request_ts         int64
user_id            int32
country_id          int8
region_id          int16
timezone_id         int8
component0         int16
component1         int16
component2         int16
component3         int16
component4         int16
component5         int16
component6         int16
component7         int16
component8         int16
component9         int32
browser             int8
browser_version    int16
os                  int8
os_version         int16
domain             int16
dtype: object

# Catbooster

In [24]:
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=6,
                           random_seed=42,
                           loss_function='Logloss',
                           od_type='Iter',
                           use_best_model=False,
                           od_wait=500,
                           verbose=100)

model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=250)

0:	learn: 0.6740152	test: 0.6740480	best: 0.6740480 (0)	total: 233ms	remaining: 3m 52s
100:	learn: 0.4905942	test: 0.4935659	best: 0.4935659 (100)	total: 8.67s	remaining: 1m 17s
200:	learn: 0.4675152	test: 0.4715094	best: 0.4715094 (200)	total: 17.4s	remaining: 1m 8s
300:	learn: 0.4541266	test: 0.4589088	best: 0.4589088 (300)	total: 25s	remaining: 58s
400:	learn: 0.4454968	test: 0.4510992	best: 0.4510992 (400)	total: 33.1s	remaining: 49.4s
500:	learn: 0.4383543	test: 0.4449036	best: 0.4449036 (500)	total: 41.1s	remaining: 40.9s
600:	learn: 0.4330156	test: 0.4407057	best: 0.4407057 (600)	total: 48.8s	remaining: 32.4s
700:	learn: 0.4283134	test: 0.4371258	best: 0.4371258 (700)	total: 56.4s	remaining: 24.1s
800:	learn: 0.4242701	test: 0.4341465	best: 0.4341465 (800)	total: 1m 3s	remaining: 15.8s
900:	learn: 0.4206356	test: 0.4317685	best: 0.4317685 (900)	total: 1m 11s	remaining: 7.87s
999:	learn: 0.4174744	test: 0.4298636	best: 0.4298636 (999)	total: 1m 19s	remaining: 0us

bestTest = 0.42

In [25]:
# Получение лучших метрик
best_metrics = model.get_best_score()

print("Лучшие метрики:")
for metric_name, value in best_metrics.items():
    print(f"{metric_name}: {value}")

# Метрики на валидационной выборке
y_pred_proba = model.predict_proba(X_val)[:, 1]
y_pred_class = model.predict(X_val)

print("\nМетрики на валидационной выборке:")
print(f"AUC-ROC: {roc_auc_score(y_val, y_pred_proba)}")
print(f"Precision: {precision_score(y_val, y_pred_class)}")
print(f"Recall: {recall_score(y_val, y_pred_class)}")
print(f"F1-score: {f1_score(y_val, y_pred_class)}")

Лучшие метрики:
learn: {'Logloss': 0.4174744484041034}
validation: {'Logloss': 0.4298635538772724}

Метрики на валидационной выборке:
AUC-ROC: 0.8807628946865991
Precision: 0.8217551709250756
Recall: 0.8032648520074428
F1-score: 0.8124048152567199


Лучшие метрики:

learn: {'Logloss': 0.4280640755209506}

validation: {'Logloss': 0.44086690373741905}

Метрики на валидационной выборке:

AUC-ROC: 0.8748066313268189

Precision: 0.8093141438435876

Recall: 0.8003874853733863

F1-score: 0.8048260630551564

AUC-ROC (Area Under ROC Curve): 0.8521745121808912


Значение выше 0.8 указывает на хорошую способность модели различать положительные и отрицательные примеры.

Precision: 0.7825230233136978

Значение выше 0.75 показывает, что модель довольно точна в своих предсказаниях о положительных классах.

Recall: 0.7355768293396244

Значение около 0.7 указывает на хорошую чувствительность модели к положительным классам.

F1-score: 0.7583240369443524

Значение между 0.7 и 0.8 показывает хороший баланс между precision и recall.

#  Рандомный лес

In [26]:
# Создание модели Random Forest
rf_model = RandomForestClassifier(
    n_estimators=1000,
    max_depth=6,
    random_state=42,
    verbose=100,
    n_jobs=-1  # Используем все доступные ядра процессора
)


In [40]:

# Обучение модели
rf_model.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.1s

building tree 14 of 1000[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.4s

building tree 15 of 1000
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    2.4s
building tree 16 of 1000
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.4s
building tree 17 of 1000
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.5s
building tree 18 of 1000
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    2.6s
building tree 19 of 1000[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    2.7s

building tree 20 o

RandomForestClassifier(max_depth=6, n_estimators=1000, n_jobs=-1,
                       random_state=42, verbose=100)

In [28]:

# Оценка модели на валидационной выборке
rf_val_pred = rf_model.predict(X_val)

# Вычисление метрик для Random Forest
rf_accuracy = accuracy_score(y_val, rf_val_pred)
rf_log_loss = log_loss(y_val, rf_model.predict_proba(X_val))

print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"Random Forest Log Loss: {rf_log_loss}")

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   7 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  11 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  13 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  15 tasks      | elaps

Random Forest Accuracy: 0.7494325094884413

Random Forest Log Loss: 0.5459558138235858

Log Loss ниже 0.6, что уже является неплохим показателем, особенно в комплексе с accuracy выше 0.7. Это говорит о том, что в целом со своей задачей модель спраляется

#  Log R

In [37]:
# Создаем модель Logistic Regression
lr_model = LogisticRegression(random_state=42,
    solver='lbfgs',  # или 'newton-cg', 'sag' или 'saga', 'lbfgs'
    max_iter=5000,  # увеличение максимального числа итераций
    tol=1e-6,     # уменьшение погрешности остановки
    C=0.1           # регуляризация (измените значение по необходимости)
)
# Обучение модели на тренировочной выборке
lr_model.fit(X_train, y_train)


LogisticRegression(C=0.1, max_iter=5000, random_state=42, tol=1e-06)

In [38]:
# Оценка модели на валидационной выборке
lr_val_pred = lr_model.predict(X_val)


In [39]:
# Вычисление метрик для Logistic Regression
lr_accuracy = accuracy_score(y_val, lr_val_pred)
lr_log_loss = log_loss(y_val, lr_model.predict_proba(X_val))

print(f"Logistic Regression Accuracy: {lr_accuracy}")
print(f"Logistic Regression Log Loss: {lr_log_loss}")

Logistic Regression Accuracy: 0.6267456008135543
Logistic Regression Log Loss: 0.644771033660341


Logistic Regression Accuracy: 0.6191820872755007

Logistic Regression Log Loss: 0.6546892136524579 

Log Loss выше 0.6, довольно высокое значение, значит модель плохо справляется с задачей

                                                Сравним метрики Random Forest и CatBoost

Log Loss:

CatBoost имеет лучшие значения Log Loss на обоих этапах обучения и валидации.

AUC-ROC:

CatBoost имеет значительно более высокое значение AUC-ROC (0.8807628946865991), что указывает на отличную способность модели различать положительные и отрицательные примеры.

Precision и Recall:
CatBoost показывает хорошие значения Precision (0.8217551709250756) и Recall (0.8032648520074428).

Эти значения близки к Random Forest, но CatBoost демонстрирует лучший баланс между precision и recall.

F1-score:

CatBoost имеет немного выше F1-score (0.8124048152567199), что подтверждает хороший баланс между precision и recall.

Accuracy:

Random Forest показывает более высокую точность (0.7492599923729275), но эта метрика может быть менее информативной в случае несбалансированных данных.

# Выводы

Из всех моделей наилучщий результат показала модель CatBooster, а наихудший логарифмическая регрессия.

- CatBoost демонстрирует более низкие значения Log Loss на обоих этапах обучения и валидации, что указывает на лучшую способность модели прогнозировать вероятности классов.

- CatBoost имеет значительно более высокое значение AUC-ROC, что говорит о его хорошей способности различать положительные и отрицательные примеры.

- CatBoost показывает хороший баланс между precision и recall, подтверждаемый высокой F1-score.

- Random Forest имеет немного выше точность, но эта метрика может быть менее информативной для задач с несбалансированными данными.

Обе модели показывают хорошие результаты по большинству метрик, но CatBoost демонстрирует более стабильную производительность по всем параметрам.
Учитывая комплексный анализ всех метрик, можно заключить, что CatBoost выглядит как более эффективная модель для данной задачи классификации. 
